In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path

CURRENT_DIRECTORY_NOTEBOOK = None


def intitate_notebook():
    load_dotenv()
    global CURRENT_DIRECTORY_NOTEBOOK
    if CURRENT_DIRECTORY_NOTEBOOK is None:
        os.chdir(os.getenv("PROJECT_BASE_PATH"))
        CURRENT_DIRECTORY_NOTEBOOK = Path(os.getcwd())
        print("Current directory for notebook: ", CURRENT_DIRECTORY_NOTEBOOK)
    else:
        print(
            "Current directory for notebook is already set: ",
            CURRENT_DIRECTORY_NOTEBOOK,
        )

intitate_notebook()

from src.utils.common import clear_gc
clear_gc()

TRAIN_PARQUET_FOLDER_PATH = (
    "raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train"
)

CLEANED_DATASET_DIRECTORY_PATH = Path("datasets/cleaned_data")

Current directory for notebook:  /Users/shirshmall/Personal_Drive/Credit_Risk_Model_Project


In [2]:
from src.utils.common import import_parquet_file, import_and_concatenate_parquet_files
from src.data_cleaning import get_columns_high_missing_values, clean_credit_bureau_dataset, clean_tax_registry_dataset, clean_static_cb_dataset, clean_static_dataset, clean_person_dataset, clean_previous_applications_dataset, clean_base_dataset, get_max_per_group, reduce_mem_usage, set_table_dtypes
from pyspark.sql.functions import countDistinct

In [ ]:
# Run Above

### Data Info
- Data Name: Base
- Relevant for Training: Yes

In [ ]:
base_df, spark = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("train_base.parquet")
    ),
    app_name="Base_Data_Cleaning_Train",
)

# base_df.show()

In [ ]:
base_df = clean_base_dataset(base_df)

In [ ]:
# SAVE the data in .parquet file # Coalesce to a single partition
base_df.coalesce(1).write.parquet(
    str(CURRENT_DIRECTORY_NOTEBOOK / CLEANED_DATASET_DIRECTORY_PATH) + "/base_data" 
)

In [ ]:
spark.stop()

### Data Info
- Data Name: Previous Applications (applprev)
- Depth: 1
- Source: Internal data source
- Relevant for Training: Yes

In [ ]:
train_applprev_df, spark = import_and_concatenate_parquet_files(
    data_file_names=["train_applprev_1_0.parquet", "train_applprev_1_1.parquet"],
    app_name="Previous_Application_Train_Dataset_Cleaning",
    base_path=TRAIN_PARQUET_FOLDER_PATH,
)

train_applprev_df.sort("case_id", "num_group1")
train_applprev_df.show(5)

In [ ]:
train_applprev_df = clean_previous_applications_dataset(train_applprev_df)

In [ ]:
print(
    "Total Number of Row: ", 
    train_applprev_df.count(),
    "\nTotal Number of Unique Case ID: ",
    train_applprev_df.select(countDistinct("case_id")).collect()[0][0],
)

train_applprev_df.show()

In [ ]:
# SAVE the data in .parquet file # Coalesce to a single partition
train_applprev_df.coalesce(1).write.parquet(
    str(CURRENT_DIRECTORY_NOTEBOOK / CLEANED_DATASET_DIRECTORY_PATH) + "/previous_applications_data" 
)

In [ ]:
spark.stop()


### Data Info
- Data Name: Credit Bureau (credit_bureau)
- Depth: 1
- Source: External data source, Credit bureau provider A and B
- Relevant for Training: Yes

##### Provider A

In [3]:
credit_bureau_df, spark = import_and_concatenate_parquet_files(
    data_file_names=[
        "train_credit_bureau_a_1_0.parquet", 
        "train_credit_bureau_a_1_1.parquet", 
        "train_credit_bureau_a_1_2.parquet", 
        "train_credit_bureau_a_1_3.parquet"],
    app_name="Credit_Bureau_A_Train_Dataset_Cleaning",
    base_path=TRAIN_PARQUET_FOLDER_PATH,
)
# credit_bureau_df.sort("case_id", "num_group1")
credit_bureau_df = set_table_dtypes(credit_bureau_df)
credit_bureau_df.show(10)
print(
    "Total Number of Row: ", 
    credit_bureau_df.count(),
    "\nTotal Number of Unique Case ID: ",
    credit_bureau_df.select(countDistinct("case_id")).collect()[0][0],
)

2024-07-25 12:46:43.990 | INFO     | src.utils.common:import_and_concatenate_parquet_files:57 - Initializing SparkSession
24/07/25 12:46:45 WARN Utils: Your hostname, Shirshs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.83 instead (on interface en0)
24/07/25 12:46:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/25 12:46:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-07-25 12:46:45.977 | INFO     | src.utils.common:import_and_concatenate_parquet_files:70 - Reading file train_credit_bureau_a_1_0.parquet with app name Credit_Bureau_A_Train_Dataset_Cleaning
2024-07-25 12:46:47.998 | INFO     | src.utils.common:import_and_concatenate_parquet_files:72 - File Loaded - Rows Count: 4108212
2024-07-25 12:46:48.002 | INFO 

+-------+------------------------+-----------------------+-------------------------+--------------------------+---------------+---------------+--------------------+------------+------------+------------------+------------------+--------------------+--------------------+--------------------+-----------------+---------------+----------------+-----------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------------+-------------------------+----------------+----------------+-----------------+----------------+---------------+-----------------------+-----------------------+----------------+----------------+----------+------------------------+------------------------+-------------------+-------------------+---------------------------+--------------------------+-----------------------------+-----------------------------+-------------------------------+-------------------------------+--------------------------+--------------------------+

Total Number of Row:  15940537 
Total Number of Unique Case ID:  1386273


24/07/25 12:46:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
# missing_values_dict = get_columns_high_missing_values( credit_bureau_df, threshold=10)
# credit_bureau_df = credit_bureau_df.drop(*list(missing_values_dict.keys()))
# credit_bureau_df.show(5)

#### Saving and Load the dataframe because it is cause 'Out of Memory' issue. - Needs Restart and Run separately

In [ ]:
grouped_max_train_applprev_df = get_max_per_group(
    df=credit_bureau_df, group_by_column="case_id", join_indicator=False,
    columns_to_skip=["dpdmaxdatemonth_442T", "dpdmaxdatemonth_89T", "dpdmaxdateyear_596T", "dpdmaxdateyear_896T" ]
)

In [ ]:
# SAVE the data in .parquet file # Coalesce to a single partition
grouped_max_train_applprev_df.coalesce(1).write.parquet(
    str(CURRENT_DIRECTORY_NOTEBOOK / CLEANED_DATASET_DIRECTORY_PATH) + "/credit_bureau_data_temp_group_max" 
)

In [ ]:
print(
    "Total Number of Row: ", 
    grouped_max_train_applprev_df.count(),
    "\nTotal Number of Unique Case ID: ",
    grouped_max_train_applprev_df.select(countDistinct("case_id")).collect()[0][0],
)

grouped_max_train_applprev_df.show(10)

In [ ]:
spark.stop()

***

In [4]:
credit_bureau_df = clean_credit_bureau_dataset(credit_bureau_df)

100%|██████████| 79/79 [00:24<00:00,  3.28it/s]
2024-07-25 12:47:46.736 | DEBUG    | src.data_cleaning:process_group_credit_bureau_dataset:412 - Group 0: Total Number of Rows: 1386273, Total Number of Unique Case ID: 1386273
100%|██████████| 79/79 [00:22<00:00,  3.58it/s]
2024-07-25 12:48:12.155 | DEBUG    | src.data_cleaning:process_group_credit_bureau_dataset:412 - Group 1: Total Number of Rows: 1386081, Total Number of Unique Case ID: 1386081


In [5]:
credit_bureau_df.show(10)

print(
    "Total Number of Row: ", 
    credit_bureau_df.count(),
    "\nTotal Number of Unique Case ID: ",
    credit_bureau_df.select(countDistinct("case_id")).collect()[0][0],
)

+-------------------------------------+--------------------------------------+---------------------------+---------------------------+-----------------------------+---------------------------+----------------------------+-------------------------------------+-------------------------------------+----------------------+------------------------------+------------------------------+----------------------------+---------------------------+-------------------------------------+--------------------------------------+---------------------------+---------------------------+----------------------------+-------------------------------------+-------------------------------------+----------------------+------------------------------+------------------------------+----------------------------+---------------------------+-------+
|classificationofcontr_13M_num_group_0|classificationofcontr_400M_num_group_0|contractst_545M_num_group_0|contractst_964M_num_group_0|debtoutstand_525A_num_group_0|debtover

Total Number of Row:  1386273 
Total Number of Unique Case ID:  1386273


In [6]:
missing_values_dict = get_columns_high_missing_values(credit_bureau_df)

100%|██████████| 27/27 [02:22<00:00,  5.29s/it]                                 


In [7]:
grouped_max_train_applprev_df, spark = import_parquet_file(
    file_path=(
        str(CURRENT_DIRECTORY_NOTEBOOK / CLEANED_DATASET_DIRECTORY_PATH) + "/credit_bureau_data_temp_group_max" 
    ),
    app_name="Credit_Bureau_Temp_Data_Cleaning_Train",
)

grouped_max_train_applprev_df.show()

2024-07-25 12:54:45.111 | INFO     | src.utils.common:import_parquet_file:37 - Starting Spark session with app name: Credit_Bureau_Temp_Data_Cleaning_Train
24/07/25 12:54:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
2024-07-25 12:54:45.134 | INFO     | src.utils.common:import_parquet_file:41 - Reading Parquet file from path: /Users/shirshmall/Personal_Drive/Credit_Risk_Model_Project/datasets/cleaned_data/credit_bureau_data_temp_group_max
2024-07-25 12:54:45.253 | INFO     | src.utils.common:import_parquet_file:44 - Parquet file loaded successfully


+-------+----------------------------+---------------------------+------------------------+----------------+----------------+---------------------+-------------------+---------------+---------------+--------------------+--------------------+---------------------+---------------------------+---------------------------+--------------------+--------------------+----------------------------+----------------------------+-----------------------+-----------------------+-------------------------------+------------------------------+---------------------------------+---------------------------------+------------------------------+------------------------------+--------------------------+--------------------------+---------------------+----------------------+-------------------------+--------------------------+-------------------------+------------------------+----------------------------------+----------------------------------+-------------------------------+---------------------------------+-

In [8]:
credit_bureau_df = credit_bureau_df.join(
    grouped_max_train_applprev_df, on="case_id", how="left"
)

In [9]:
print(
    "Number of columns: ",
    len(credit_bureau_df.columns),
    "Total Number of Row: ", 
    credit_bureau_df.count(),
    "\nTotal Number of Unique Case ID: ",
    credit_bureau_df.select(countDistinct("case_id")).collect()[0][0],
)

credit_bureau_df.show(5)

Number of columns:  77 Total Number of Row:  1386273 
Total Number of Unique Case ID:  1386273


+-------+-------------------------------------+--------------------------------------+---------------------------+---------------------------+-----------------------------+---------------------------+----------------------------+-------------------------------------+-------------------------------------+----------------------+------------------------------+------------------------------+----------------------------+---------------------------+-------------------------------------+--------------------------------------+---------------------------+---------------------------+----------------------------+-------------------------------------+-------------------------------------+----------------------+------------------------------+------------------------------+----------------------------+---------------------------+----------------------------+---------------------------+------------------------+----------------+----------------+---------------------+-------------------+---------------

In [10]:
# SAVE the data in .parquet file # Coalesce to a single partition
credit_bureau_df.coalesce(1).write.parquet(
    str(CURRENT_DIRECTORY_NOTEBOOK / CLEANED_DATASET_DIRECTORY_PATH) + "/credit_bureau_data" 
)

In [12]:
spark.stop()

### Data Info
- Data Name: Debit Card (debitcard)
- Depth: 1
- Source: Internal data source
- Relevant for Training: No

In [ ]:
debit_card_df, spark = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("train_debitcard_1.parquet")
    ),
    app_name="Debit_Card_Data_Cleaning_Train",
)

debit_card_df.show()

In [ ]:
missing_values_dict = get_columns_high_missing_values(
    debit_card_df, threshold=None, display=False
)


print(
    "Number of columns: ",
    len(debit_card_df.columns),
    "\nTotal Number of Row: ", 
    debit_card_df.count(),
    "\nTotal Number of Unique Case ID: ",
    debit_card_df.select(countDistinct("case_id")).collect()[0][0],
)

In [ ]:
spark.stop()

### Data Info
- Data Name: Deposits (deposit)
- Depth: 1
- Source: Internal data source
- Relevant for Training: No

In [ ]:
deposit_df, spark = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("train_deposit_1.parquet")
    ),
    app_name="Deposit_Data_Cleaning_Train",
)

deposit_df.show()

In [ ]:
missing_values_dict = get_columns_high_missing_values(
    deposit_df, threshold=None, display=False
)


print(
    "Number of columns: ",
    len(deposit_df.columns),
    "\nTotal Number of Row: ", 
    deposit_df.count(),
    "\nTotal Number of Unique Case ID: ",
    deposit_df.select(countDistinct("case_id")).collect()[0][0],
)

In [ ]:
spark.stop()

### Data Info
- Data Name: Other (other)
- Depth: 1
- Source: Internal data source
- Relevant for Training: No

In [ ]:
other_df, spark = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("train_other_1.parquet")
    ),
    app_name="Other_Data_Cleaning_Train",
)

other_df.show()

In [ ]:
missing_values_dict = get_columns_high_missing_values(
    other_df, threshold=None, display=False
)

print(
    "Number of columns: ",
    len(other_df.columns),
    "\nTotal Number of Row: ", 
    other_df.count(),
    "\nTotal Number of Unique Case ID: ",
    other_df.select(countDistinct("case_id")).collect()[0][0],
)

In [ ]:
spark.stop()

### Data Info
- Data Name: Person (person)
- Depth: 1
- Source: Internal data source
- Relevant for Training: Yes

In [ ]:
person_df, spark = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("train_person_1.parquet")
    ),
    app_name="Person_Data_Cleaning_Train",
)

person_df.show(10)

In [ ]:
person_df = clean_person_dataset(person_df)

In [ ]:
# SAVE the data in .parquet file # Coalesce to a single partition
person_df.coalesce(1).write.parquet(
    str(CURRENT_DIRECTORY_NOTEBOOK / CLEANED_DATASET_DIRECTORY_PATH) + "/person_data" 
)

person_df.show()

In [ ]:
spark.stop()

### Data Info
- Data Name: Static (static)
- Depth: 0
- Source: Internal data source
- Relevant for Training: Yes

In [ ]:
static_df, spark = import_and_concatenate_parquet_files(
    data_file_names=[
        "train_static_0_0.parquet", 
        "train_static_0_1.parquet", 
    ],
    app_name="Static_Train_Dataset_Cleaning",
    base_path=TRAIN_PARQUET_FOLDER_PATH,
)

print(
    "Number of columns: ",
    len(static_df.columns),
    "\nTotal Number of Row: ", 
    static_df.count(),
    "\nTotal Number of Unique Case ID: ",
    static_df.select(countDistinct("case_id")).collect()[0][0],
)

static_df.show(10)

In [ ]:
static_df = clean_static_dataset(static_df)

In [ ]:
print(
    "Number of columns: ",
    len(static_df.columns),
    "\nTotal Number of Row: ", 
    static_df.count(),
    "\nTotal Number of Unique Case ID: ",
    static_df.select(countDistinct("case_id")).collect()[0][0],
)

static_df.show(10)

In [ ]:
spark.stop()

### Data Info
- Data Name: Static Credit Bureau (static_cb)
- Depth: 0
- Source: External data source
- Relevant for Training: Yes

In [ ]:
static_cb_df, spark = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("train_static_cb_0.parquet")
    ),
    app_name="Static_CB_Data_Cleaning_Train",
)


print(
    "Number of columns: ",
    len(static_cb_df.columns),
    "\nTotal Number of Row: ", 
    static_cb_df.count(),
    "\nTotal Number of Unique Case ID: ",
    static_cb_df.select(countDistinct("case_id")).collect()[0][0],
)

static_cb_df.show(10)

In [ ]:
static_cb_df = clean_static_cb_dataset(static_cb_df)

In [ ]:
print(
    "Number of columns: ",
    len(static_cb_df.columns),
    "\nTotal Number of Row: ", 
    static_cb_df.count(),
    "\nTotal Number of Unique Case ID: ",
    static_cb_df.select(countDistinct("case_id")).collect()[0][0],
)

static_cb_df.show(10)

In [ ]:
static_cb_df.coalesce(1).write.parquet(
    str(CURRENT_DIRECTORY_NOTEBOOK / CLEANED_DATASET_DIRECTORY_PATH) + "/static_data" 
)

In [ ]:
spark.stop()

### Data Info
- Data Name: Tax Registry (tax_registry)
- Depth: 1
- Source: External data source, Tax registry provider A, B and C
- Relevant for Training: Yes

In [ ]:
tax_register_a_df, spark_a = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("train_tax_registry_a_1.parquet")
    ),
    app_name="Tax_Registry_A_Data_Cleaning_Train",
)

tax_register_b_df, spark_b = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("train_tax_registry_b_1.parquet")
    ),
    app_name="Tax_Registry_B_Data_Cleaning_Train",
)

tax_register_c_df, spark_c = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("train_tax_registry_c_1.parquet")
    ),
    app_name="Tax_Registry_C_Data_Cleaning_Train",
)

In [ ]:
from pyspark.sql.functions import lit

# Define the column mapping
column_mapping = {
    'case_id': 'case_id',
    'amount_4527230A': 'amount',
    'amount_4917619A': 'amount',
    'pmtamount_36A': 'amount',
    'name_4527232M': 'name',
    'name_4917606M': 'name',
    'employername_160M': 'name',
    'num_group1': 'num_group',
    'recorddate_4527225D': 'tax_registry_date',
    'deductiondate_4917603D': 'tax_registry_date',
    'processingdate_168D': 'tax_registry_date'
}

# Function to rename columns based on the mapping
def rename_columns_add_indicator(df, column_mapping, indicator):
    # Renaming
    for old_name, new_name in column_mapping.items():
        if old_name in df.columns:
            df = df.withColumnRenamed(old_name, new_name)
    
    # Adding Indicator
    df = df.withColumn("tax_registry_provider", lit(indicator))
    return df

# Rename columns in each DataFrame
tax_register_a_df = rename_columns_add_indicator(tax_register_a_df, column_mapping, indicator="a")
tax_register_b_df = rename_columns_add_indicator(tax_register_b_df, column_mapping, indicator="b")
tax_register_c_df = rename_columns_add_indicator(tax_register_c_df, column_mapping, indicator="c")

In [ ]:
tax_register_df = tax_register_a_df.unionByName(tax_register_b_df).unionByName(tax_register_c_df)

print(
    "Number of columns: ",
    len(tax_register_df.columns),
    "\nTotal Number of Row: ", 
    tax_register_df.count(),
    "\nTotal Number of Unique Case ID: ",
    tax_register_df.select(countDistinct("case_id")).collect()[0][0],
)

tax_register_df.show(10)

In [ ]:
tax_register_df = clean_tax_registry_dataset(tax_register_df)

In [ ]:
print(
    "Number of columns: ",
    len(tax_register_df.columns),
    "\nTotal Number of Row: ", 
    tax_register_df.count(),
    "\nTotal Number of Unique Case ID: ",
    tax_register_df.select(countDistinct("case_id")).collect()[0][0],
)

tax_register_df.show()

In [ ]:
tax_register_df.dropDuplicates(["tax_registry_provider"]).show()

In [ ]:
tax_register_df.coalesce(1).write.parquet(
    str(CURRENT_DIRECTORY_NOTEBOOK / CLEANED_DATASET_DIRECTORY_PATH) + "/tax_registry_data" 
)

In [ ]:
spark_a.stop()
spark_b.stop()
spark_c.stop()